### Multiagent Active Blockference

This notebook is an experimental exploration of multi-agent active inference. CadCAD is not used at this point.

In [87]:
import itertools
import numpy as np
import sys

# adding tools to the system path
sys.path.insert(0, '../')

In [79]:
# start with 2x2 grid
grid = list(itertools.product(range(2), repeat=2))
border = np.sqrt(len(grid)) - 1
pos_dict = {}
for i in range(0, len(grid)):
    pos_dict[i] = grid[i]
print(pos_dict)
num_agents = 2 # start with 2 agents
init_pos = [0, 3] # agents will start at positions 0 and 3

{0: (0, 0), 1: (0, 1), 2: (1, 0), 3: (1, 1)}


In [69]:
# getting the grid positions and indexes for the two agents A & B
init_A = init_pos[0]
init_B = init_pos[1]
init_A_pos = pos_dict[init_A]
init_B_pos = pos_dict[init_B]

In [85]:
# getting the preferred grid positions and indexes for the two agents A & B
# their preferred position will be the one where the other agent starts
pref_A = 3
pref_B = 0
pref_A_pos = pos_dict[pref_A]
pref_B_pos = pos_dict[pref_B]

#### Observations and States
In a single-agent environment, observations and states are both just the number of positions (because the agent can be at 4 different positions (4 states) and have 4 different observations).

Adding an extra agents adds extra complexity. We let our agents be strictly non-interacting, i.e. they cannot occupy the same position on the grid at the same time.

Both agents started with having 4 possible states, hence 4*4=16 possible states, but the restriction on non-interactivity reduces this number by the 4 positions where both agents are present, hence **number of possible states is 12**.

In [70]:
# observations and states
n_states = 12
n_observations = 12 # have to do more work on this, might be reduced assuming completely symmetric states

#### Generative Model Tensors
Now we define the tensors describing the generative model. For detailed explanations of what each tensor means/does, see:
https://pymdp-rtd.readthedocs.io/en/latest/notebooks/active_inference_from_scratch.html

In [76]:
# E vector (affordances)
E = ["UP", "DOWN", "LEFT", "RIGHT", "STAY"]

In [75]:
# A matrix
# Note: maybe multi-agent actinf does not change B but rather A & D
A = np.eye(n_observations, n_states)
print(A)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [83]:
# B matrix
# Note: B should only encode prior beliefs about !controllable! transitions between hidden states
# why/how can we assume the actions of the other agents are within controllable transitions?
B = np.zeros((len(grid), len(grid), len(E)))

for action_id, action_label in enumerate(E):

    for curr_state, grid_location in enumerate(grid):

        y, x = grid_location

        if action_label == "UP":
            next_y = y - 1 if y > 0 else y
            next_x = x
        elif action_label == "DOWN":
            next_y = y + 1 if y < border else y
            next_x = x
        elif action_label == "LEFT":
            next_x = x - 1 if x > 0 else x
            next_y = y
        elif action_label == "RIGHT":
            next_x = x + 1 if x < border else x
            next_y = y
        elif action_label == "STAY":
            next_x = x
            next_y = y
        new_location = (next_y, next_x)
        next_state = grid.index(new_location)
        B[next_state, curr_state, action_id] = 1.0
print(B)

[[[1. 0. 1. 0. 1.]
  [0. 0. 1. 0. 0.]
  [1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 1. 0.]
  [1. 0. 0. 1. 1.]
  [0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 1. 1. 0. 1.]
  [0. 0. 1. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0.]
  [0. 0. 0. 1. 0.]
  [0. 1. 0. 1. 1.]]]


In [100]:
import tools.utils as utils

# C -> preferred state

# C for agent A
C_A = utils.onehot(grid.index(pref_A_pos), len(grid)) # originally len(grid) was n_observations but that doesn't seem correct now

# C for agent B
C_B = utils.onehot(grid.index(pref_B_pos), len(grid))

print(C_A)
print(C_B)

[0. 0. 0. 1.]
[1. 0. 0. 0.]


In [102]:
# D -> initial prior

D_A = utils.onehot(grid.index(init_A_pos), len(grid)) # REVISIT: originally n_states but again did not seem correct
D_B = utils.onehot(grid.index(init_B_pos), len(grid))

print(D_A)
print(D_B)

[1. 0. 0. 0.]
[0. 0. 0. 1.]


In [15]:
# WIP
chosen_action = None
if chosen_action == 0:  # UP

    Y_new = Y - 1 if Y > 0 else Y
    X_new = X

elif chosen_action == 1:  # DOWN

    Y_new = Y + 1 if Y < agent.border else Y
    X_new = X

elif chosen_action == 2:  # LEFT
    Y_new = Y
    X_new = X - 1 if X > 0 else X

elif chosen_action == 3:  # RIGHT
    Y_new = Y
    X_new = X + 1 if X < agent.border else X

elif chosen_action == 4:  # STAY/WATCH (i.e. watch what the other agent will do)
    Y_new, X_new = Y, X